# 阶跃星辰 TTS API 开发实践指南

> 基于文档：
> 
> [https://platform.stepfun.com/docs/guide/tts](https://platform.stepfun.com/docs/guide/tts)
> 
>  的实操教程

## 1. 概述

本 Notebook 旨在帮助开发者快速上手阶跃星辰 TTS（文本转语音）API，涵盖环境准备、快速生成音频、功能参数调整及典型应用场景实践，助力将语音交互能力集成到各类应用中。

## 2. 环境准备

### 2.1 安装依赖库

需安装 `requests` 用于 API 调用，`IPython` 用于音频播放（Jupyter 环境自带，若缺失可手动安装）。

In [ ]:
# 安装必要依赖
!pip install requests -q

### 2.2 核心前置条件

1.  获取阶跃星辰 API 密钥（`STEP_API_KEY`）：需通过阶跃星辰开发者平台申请
2.  确认网络环境可访问 API 域名：`api.stepfun.com`

In [1]:
import os
# 配置 API 密钥
STEP_API_KEY = os.getenv("STEPFUN_API_KEY")
BASE_URL = "https://api.stepfun.com/v1/audio/speech"

## 3. 快速开始：生成第一段音频

基于文档提供的基础示例，通过 Python 实现音频生成、保存与播放。

In [8]:
import requests

def generate_basic_audio(api_key, output_path="./media/06_step_basic.mp3"):
    """生成基础示例音频"""
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    # 请求参数（参考文档基础示例）
    payload = {
        "model": "step-tts-mini",  # 选用轻量模型
        "input": "智能阶跃，十倍每一个人的可能",  # 待转换文本
        "voice": "cixingnansheng"  # 音色：磁性男声
    }
    
    try:
        # 发送 POST 请求
        response = requests.post(BASE_URL, json=payload, headers=headers)
        response.raise_for_status()  # 捕获 HTTP 错误

        print(response.text)
        # 保存音频文件
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"音频已生成并保存至：{output_path}")
        return output_path
    
    except Exception as e:
        print(f"生成失败：{str(e)}")
        return None

# 执行生成
audio_path = generate_basic_audio(STEP_API_KEY)

ID3    TSSE     Lavf58.76.100 TXXX  {   AIGC {"Label":"1","ContentProducer":"001191310104MACE0YX63918010","ProduceID":"2fc9503c9bd93c13eab69c8b13883be8","ReservedCode1":"{\"SecurityData\":{\"Type\":\"TC260PG\",\"Version\":1,\"PubSD\":[{\"Type\":\"DS\",\"AlgID\":\"1.2.156.10197.1.501\",\"TBSData\":{\"Type\":\"LabelMetaData\"},\"Signature\":\"30450220275670A8E0D80F26BAE985D0D802C3136C3EF6B62533865C0FADF5B485C484BF022100A87D8897FF49D329026030FE5FB8800A1C434BBEB44E59C4EA16FF1385A84E7B\"},{\"Type\":\"PubKey\",\"AlgID\":\"1.2.156.10197.1.501\",\"KeyValue\":\"3059301306072A8648CE3D020106082A811CCF5501822D034200045728F08C58D9C72150A705693EF7337859F8FF216E802630041AFAE5C036F0A67CB1FDF8C57F5F8B623284AE09E4E545464B7140B26BBDA51BAE965F854C57AA\"}]}}","ContentPropagator":"001191310104MACE0YX63918010","PropagateID":"2fc9503c9bd93c13eab69c8b13883be8","ReservedCode2":"{\"SecurityData\":{\"Type\":\"TC260PG\",\"Version\":1,\"PubSD\":[{\"Type\":\"DS\",\"AlgID\":\"1.2.156.10197.1.501\",\"TBSData\":{

In [11]:
# 播放生成的音频（仅支持 Jupyter 环境）
from IPython.display import Audio

if audio_path:
    Audio(audio_path)

## 4. 核心功能详解

### 4.1 模型与音色选择

阶跃星辰提供两种 TTS 模型，音色支持各有侧重，可根据场景需求选择。

#### 模型对比表

| 模型名称           | 音色数量 | 核心特点        | 适用场景             |
| -------------- | ---- | ----------- | ---------------- |
| step-tts-vivid | 4 种  | 优质生动，场景适配性强 | 客服外呼、陪伴场景等       |
| step-tts-mini  | 19 种 | 轻量多样，支持音色复刻 | 游戏 NPC、有声书等个性化场景 |

#### 常用音色调用示例

In [6]:
# step-tts-vivid 音色示例（干练女声-客服场景）
payload_vivid = {
    "model": "step-tts-vivid",
    "input": "您好，请问有什么可以帮您？",
    "voice": "ganliannvsheng"  # 干练女声 Voice ID
}

# step-tts-mini 音色示例（冷艳御姐-游戏NPC）
payload_mini = {
    "model": "step-tts-mini",
    "input": "此路不通，速速退去！",
    "voice": "lengyanyujie"  # 冷艳御姐 Voice ID
}

### 4.2 输出格式与语言设置

#### 支持的输出格式

默认输出 `mp3` 格式，可通过 `response_format` 参数指定其他格式：

*   可选格式：`wav`、`mp3`、`flac`、`opus`

#### 支持的输出语言

*   中文、英文、中英混合、日语

In [7]:
def generate_custom_audio(api_key, text, model="step-tts-mini", voice="cixingnansheng",
                         response_format="wav", language="zh", output_path="custom_audio.wav"):
    """生成自定义格式/语言的音频"""
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
        "model": model,
        "input": text,
        "voice": voice,
        "response_format": response_format  # 指定输出格式
        # 语言参数默认适配文本，复杂场景可显式指定
    }
    
    try:
        response = requests.post(BASE_URL, json=payload, headers=headers)
        response.raise_for_status()
        
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"自定义音频已保存至：{output_path}（格式：{response_format}）")
        return output_path
    
    except Exception as e:
        print(f"生成失败：{str(e)}")
        return None

# 示例：生成英文+wav格式音频（元气男声）
en_audio_path = generate_custom_audio(
    api_key=STEP_API_KEY,
    text="Stepfun AI, empowering possibilities.",
    voice="yuanqinansheng",
    response_format="wav",
    output_path="./media/06_english_audio.wav"
)

自定义音频已保存至：./media/06_english_audio.wav（格式：wav）


In [8]:
# 播放自定义音频
if en_audio_path:
    Audio(en_audio_path)

### 4.3 音量与语速调整

通过 `volume` 和 `speed` 参数自定义音频效果：

*   `volume`：音量系数（0.1~2.0，默认 1.0），0.1=10% 音量，2.0=200% 音量
*   `speed`：语速系数（0.5~2.0，默认 1.0），0.5 = 半速，2.0 = 两倍速

In [20]:
def generate_adjusted_audio(api_key, text, volume=1.0, speed=1.0, output_path="./media/06_adjusted_audio.mp3"):
    """生成调整音量和语速的音频"""
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
        "model": "step-tts-mini",
        "input": text,
        "voice": "boyinnansheng",  # 播音男声
        "volume": volume,  # 音量参数
        "speed": speed     # 语速参数
    }
    
    try:
        response = requests.post(BASE_URL, json=payload, headers=headers)
        response.raise_for_status()
        
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"调整后音频已保存：音量{volume*100}%，语速{speed*100}%")
        return output_path
    
    except Exception as e:
        print(f"生成失败：{str(e)}")
        return None

# 示例：生成150%音量、0.8倍速的音频
adjusted_audio = generate_adjusted_audio(
    api_key=STEP_API_KEY,
    text="欢迎使用阶跃星辰TTS服务，这里是语速和音量调整示例。",
    volume=1.5,
    speed=0.8,
    output_path="./media/06_volume_speed_demo.mp3"
)

调整后音频已保存：音量150.0%，语速80.0%


In [ ]:
# 播放调整后音频
if adjusted_audio:
    Audio(adjusted_audio)

## 5. 典型应用场景实践

### 5.1 场景 1：有声书片段生成

需求：将小说文本转换为适配阅读场景的音频，选用温和音色 + 适中语速。

In [15]:
def generate_audiobook_segment(api_key, novel_text, output_path="./media/06_audiobook_segment.mp3"):
    """生成有声书片段"""
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
        "model": "step-tts-mini",
        "input": novel_text,
        "voice": "ruyananshi",  # 儒雅男士（适合小说旁白）
        "speed": 0.9,  # 稍慢语速，提升听感
        "volume": 1.2  # 略高音量，清晰可辨
    }
    
    try:
        response = requests.post(BASE_URL, json=payload, headers=headers)
        response.raise_for_status()
        
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"有声书片段已生成：{output_path}")
        return output_path
    
    except Exception as e:
        print(f"生成失败：{str(e)}")
        return None

# 小说文本示例（片段）
novel_text = "夜幕像一块巨大的黑丝绒，缓缓覆盖了整座城市。巷子里的路灯次第亮起，在湿漉漉的地面上投下长长的光影。"

audiobook_path = generate_audiobook_segment(STEP_API_KEY, novel_text)

有声书片段已生成：./media/06_audiobook_segment.mp3


In [ ]:
# 播放有声书片段
if audiobook_path:
    Audio(audiobook_path)

### 5.2 场景 2：游戏 NPC 语音生成

需求：为战斗型 NPC 生成台词，选用冷硬音色 + 稍快语速，增强紧张感。

In [17]:
def generate_game_npc_voice(api_key, npc_lines, output_path="./media/06_npc_voice.mp3"):
    """生成游戏NPC语音"""
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
        "model": "step-tts-mini",
        "input": npc_lines,
        "voice": "shenchennanyin",  # 深沉男音（适合战斗NPC）
        "speed": 1.3,  # 稍快语速，增强气势
        "volume": 1.5  # 高音量，突出战斗氛围
    }
    
    try:
        response = requests.post(BASE_URL, json=payload, headers=headers)
        response.raise_for_status()
        
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"NPC语音已生成：{output_path}")
        return output_path
    
    except Exception as e:
        print(f"生成失败：{str(e)}")
        return None

# NPC战斗台词
npc_lines = "入侵者！休想越过此地半步！拿出你的武器，让我看看你的实力！"

npc_audio_path = generate_game_npc_voice(STEP_API_KEY, npc_lines)

NPC语音已生成：./media/06_npc_voice.mp3


In [ ]:
# 播放NPC语音
if npc_audio_path:
    Audio(npc_audio_path)

## 6. 常见问题（FAQ）解答


1.  **音频所有权**：生成的音频归开发者所有，但建议在产品中注明 “AI 生成” 以提示用户。
2.  **音量调整**：通过 API 请求的 `volume` 参数设置（0.1~2.0），详见 4.3 节示例。
3.  **语速调整**：通过 API 请求的 `speed` 参数设置（0.5~2.0），详见 4.3 节示例。
4.  **音色复刻**：`step-tts-mini` 支持自定义音色复刻，需通过开发者平台额外申请该能力。

## 7. 扩展建议


1.  **批量生成**：对于有声书、NPC 台词等批量需求，可封装循环逻辑批量调用 API。
2.  **错误处理**：生产环境中建议增加超时重试、请求频率控制（参考平台限流规则）。
3.  **上下文结合**：结合大模型生成文本后，通过分句处理再调用 TTS，优化长文本朗读效果。
4.  **音色管理**：对于多场景应用，可维护音色 ID 映射表，动态匹配场景需求。
